In [1]:
import cv2
import copy
from matplotlib import pyplot as plt
import numpy as np
import math

img = cv2.imread('lena.bmp')

### 1. Laplace Mask
- 1 (0, 1, 0, 1, -4, 1, 0, 1, 0): 15
- 2 (1, 1, 1, 1, -8, 1, 1, 1, 1)

In [2]:
def laplace(src_img, mask):
    pos=[[-1,-1],[-1,0],[-1,1],[0,-1],[0,0],[0,1],[1,-1],[1,0],[1,1]]
    mask1=[0,1,0,1,-4,1,0,1,0]
    mask2=[1,1,1,1,-8,1,1,1,1]
    out=np.zeros((512,512,3))
    for i in range(src_img.shape[0]):
        for j in range(src_img.shape[1]):
            mag=0
            for k in range(len(pos)):
                if (i+pos[k][0]<src_img.shape[0]) and (j+pos[k][1]<src_img.shape[1]):
                    if mask==1:
                        mag+=mask1[k]*src_img[i+pos[k][0],j+pos[k][1],1]
                    elif mask==2:
                        mag+=mask2[k]*src_img[i+pos[k][0],j+pos[k][1],1]/3
            out[i,j]=mag
    return out

In [16]:
def zerocross_update(src_img, threshold):
    pos=[[-1,-1],[-1,0],[-1,1],[0,-1],[0,1],[1,-1],[1,0],[1,1]]
    out=np.zeros((512,512,3))
    for i in range(src_img.shape[0]):
        for j in range(src_img.shape[1]):
            if src_img[i,j,1]<threshold:
                out[i,j]=1
                """
                for k in range(len(pos)):
                    if (i+pos[k][0]<src_img.shape[0]) and (j+pos[k][1]<src_img.shape[1]) and (src_img[i+pos[k][0],j+pos[k][1],1]==-1):
                        out[i,j]=0
                        break
                        
            elif src_img[i,j,1]==-1:
                #out[i,j]=0
                for k in range(len(pos)):
                    if (i+pos[k][0]<src_img.shape[0]) and (j+pos[k][1]<src_img.shape[1]) and (src_img[i+pos[k][0],j+pos[k][1],1]==1):
                        out[i,j]=0
                        break
            """
    return out

In [6]:
lap1=laplace(img,1)
#lap1z=zerocross_update(lap1,15)
#lap1=zerocross_update(lap1)
#lap2=laplace(img,15,2)
#lap2=zerocross(lap2)

In [7]:
cv2.imwrite('laplace1.bmp',(lap1 < int(15)) * 255)

True

In [8]:
lap2=laplace(img,2)
#lap2z=zerocross_update(lap2,15)

In [9]:
cv2.imwrite('laplace2.bmp',(lap2 < int(15)) * 255)

True

### 2. Minimum variance Laplacian: 20

In [17]:
def min_laplace(src_img):
    pos=[[-1,-1],[-1,0],[-1,1],[0,-1],[0,0],[0,1],[1,-1],[1,0],[1,1]]
    mask=[2,-1,2,-1,-4,-1,2,-1,2]
    out=np.zeros((512,512,3))
    for i in range(src_img.shape[0]):
        for j in range(src_img.shape[1]):
            mag=0
            for k in range(len(pos)):
                if (i+pos[k][0]<src_img.shape[0]) and (j+pos[k][1]<src_img.shape[1]):
                    mag+=mask[k]*src_img[i+pos[k][0],j+pos[k][1],1]/3
            out[i,j]=mag
    return out

In [18]:
min_lap=min_laplace(img)

In [19]:
cv2.imwrite('min_laplace.bmp',(min_lap < int(20)) * 255)

True

### 3. Laplace of Gaussian: 3000

In [20]:
img = cv2.imread('lena.bmp',0)

In [21]:
def convolve(A, B):
    assert A.shape == B.shape
    value = 0
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            value += (A[i, j] * B[B.shape[0] - i - 1, B.shape[1] - j - 1])
    return value

In [22]:
def convolve_img(img, kernel):
    img_new = np.zeros((img.shape[0] - kernel.shape[0] + 1, img.shape[1] - kernel.shape[1] + 1))
    for i in range(img_new.shape[0]):
        for j in range(img_new.shape[1]):
            img_new[i, j] = convolve(img[i:i+kernel.shape[0], j:j+kernel.shape[1]], kernel)
    return img_new

In [23]:
k = np.array([
            [0, 0, 0, -1, -1, -2, -1, -1, 0, 0, 0],
            [0, 0, -2, -4, -8, -9, -8, -4, -2, 0, 0],
            [0, -2, -7, -15, -22, -23, -22, -15, -7, -2, 0],
            [-1, -4, -15, -24, -14, -1, -14, -24, -15, -4, -1],
            [-1, -8, -22, -14, 52, 103, 52, -14, -22, -8, -1],
            [-2, -9, -23, -1, 103, 178, 103, -1, -23, -9, -2],
            [-1, -8, -22, -14, 52, 103, 52, -14, -22, -8, -1],
            [-1, -4, -15, -24, -14, -1, -14, -24, -15, -4, -1],
            [0, -2, -7, -15, -22, -23, -22, -15, -7, -2, 0],
            [0, 0, -2, -4, -8, -9, -8, -4, -2, 0, 0],
            [0, 0, 0, -1, -1, -2, -1, -1, 0, 0, 0]
        ])
img_ans = convolve_img(img, k)
cv2.imwrite('lapofGau.bmp', (img_ans < int(3000)) * 255)

True

### 4. Difference of Gaussian: 1

In [25]:
k = np.array([
            [-1, -3, -4, -6, -7, -8, -7, -6, -4, -3, -1],
            [-3, -5, -8, -11, -13, -13, -13, -11, -8, -5, -3],
            [-4, -8, -12, -16, -17, -17, -17, -16, -12, -8, -4],
            [-6, -11, -16, -16, 0, 15, 0, -16, -16, -11, -6],
            [-7, -13, -17, 0, 85, 160, 85, 0, -17, -13, -7],
            [-8, -13, -17, 15, 160, 283, 160, 15, -17, -13, -8],
            [-7, -13, -17, 0, 85, 160, 85, 0, -17, -13, -7],
            [-6, -11, -16, -16, 0, 15, 0, -16, -16, -11, -6],
            [-4, -8, -12, -16, -17, -17, -17, -16, -12, -8, -4],
            [-3, -5, -8, -11, -13, -13, -13, -11, -8, -5, -3],
            [-1, -3, -4, -6, -7, -8, -7, -6, -4, -3, -1],
        ])
img_ans = convolve_img(img, k)
cv2.imwrite('diffofGau.bmp', (img_ans >= int(1)) * 255)

True